<a href="https://colab.research.google.com/github/Ranzor234/Komputasi-Intelegensia/blob/main/Week7Task_AthallaKarenza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

Load Model

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Check if GPU is available and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (

Sentiment Analysis Function

In [3]:
def predict_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the predicted class to sentiment
    sentiments = {0: "neutral", 1: "positive", 2: "negative"}
    return sentiments[predicted_class]


Test

In [4]:
# Example: Predict the sentiment of a sentence
sentence = "The stock price surged after the announcement of new product innovations."

print(f"Input: {sentence}")
print(f"Predicted Sentiment: {predict_sentiment(sentence)}")


Input: The stock price surged after the announcement of new product innovations.
Predicted Sentiment: negative


Test in synthetic dataset and accuracy test

In [5]:
# Synthetic dataset (example financial news)
synthetic_data = [
    ("The company reported record profits for the quarter.", "positive"),
    ("The CEO announced a massive layoff due to declining sales.", "negative"),
    ("Analysts predict a stable market for the next quarter.", "neutral"),
    ("Investors are excited about the company's new partnerships.", "positive"),
    ("There are growing concerns over the company's financial stability.", "negative"),
]

# Map sentiment to labels for comparison
label_mapping = {"neutral": 0, "positive": 1, "negative": 2}

# Evaluate model accuracy
correct = 0

for text, actual_label in synthetic_data:
    predicted_label = predict_sentiment(text)
    print(f"Text: {text}\nPredicted: {predicted_label}, Actual: {actual_label}\n")
    if predicted_label == actual_label:
        correct += 1

accuracy = correct / len(synthetic_data)
print(f"Accuracy on synthetic dataset: {accuracy * 100:.2f}%")


Text: The company reported record profits for the quarter.
Predicted: negative, Actual: positive

Text: The CEO announced a massive layoff due to declining sales.
Predicted: neutral, Actual: negative

Text: Analysts predict a stable market for the next quarter.
Predicted: negative, Actual: neutral

Text: Investors are excited about the company's new partnerships.
Predicted: negative, Actual: positive

Text: There are growing concerns over the company's financial stability.
Predicted: neutral, Actual: negative

Accuracy on synthetic dataset: 0.00%


Implementation of attention transformer

In [6]:
import torch.nn as nn

class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionLayer, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, hidden_states):
        # Apply attention to hidden states
        weights = torch.softmax(self.attention(hidden_states), dim=1)
        context_vector = torch.sum(weights * hidden_states, dim=1)
        return context_vector

# Load model with attention layer
class SentimentModelWithAttention(nn.Module):
    def __init__(self, base_model):
        super(SentimentModelWithAttention, self).__init__()
        self.base_model = base_model
        self.attention = AttentionLayer(hidden_size=768)  # Adjust for model size
        self.classifier = nn.Linear(768, 3)  # 3 output classes

    def forward(self, input_ids, attention_mask):
        # Get hidden states from base model
        outputs = self.base_model.roberta(input_ids, attention_mask=attention_mask).last_hidden_state
        # Apply attention and classifier
        context_vector = self.attention(outputs)
        logits = self.classifier(context_vector)
        return logits

# Initialize model with attention
attention_model = SentimentModelWithAttention(model).to(device)


Attention Transformer and Comparison between Attention vs Without Attention

In [7]:
# Redefine the prediction function to use the attention model
def predict_sentiment_with_attention(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        logits = attention_model(**inputs)

    predicted_class = torch.argmax(logits, dim=-1).item()
    sentiments = {0: "neutral", 1: "positive", 2: "negative"}
    return sentiments[predicted_class]

# Evaluate accuracy using the attention model
correct_attention = 0

for text, actual_label in synthetic_data:
    predicted_label = predict_sentiment_with_attention(text)
    print(f"Text: {text}\nPredicted: {predicted_label}, Actual: {actual_label}\n")
    if predicted_label == actual_label:
        correct_attention += 1

accuracy_attention = correct_attention / len(synthetic_data)
print(f"Accuracy with Attention Transformer: {accuracy_attention * 100:.2f}%")

# Compare accuracy before and after attention
print(f"Original Model Accuracy: {accuracy * 100:.2f}%")
print(f"Attention Transformer Model Accuracy: {accuracy_attention * 100:.2f}%")


Text: The company reported record profits for the quarter.
Predicted: negative, Actual: positive

Text: The CEO announced a massive layoff due to declining sales.
Predicted: negative, Actual: negative

Text: Analysts predict a stable market for the next quarter.
Predicted: positive, Actual: neutral

Text: Investors are excited about the company's new partnerships.
Predicted: positive, Actual: positive

Text: There are growing concerns over the company's financial stability.
Predicted: negative, Actual: negative

Accuracy with Attention Transformer: 60.00%
Original Model Accuracy: 0.00%
Attention Transformer Model Accuracy: 60.00%
